In [28]:
import numpy as np
import math
import pandas as pd
from math import log
data = pd.read_csv('泰坦尼克号数据.csv')
data = data.replace('',np.NaN)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [29]:
data = data.drop(columns='Cabin')
data = data.drop(columns='PassengerId')
data = data.drop(columns='Name')
data = data.drop(columns='Ticket')
data = data.drop(columns='Fare')
null_all = data.isnull().sum()
null_all

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Embarked      2
dtype: int64

In [30]:
data.loc[:,"Age"] = data.fillna(data.loc[:,"Age"].median())

In [31]:
copy = data.copy()
copy.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [32]:
from sklearn.preprocessing import LabelEncoder

data.iloc[:,2] = LabelEncoder().fit_transform(data.iloc[:,2])
data.head()


,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,1,22.0,1,0,S
1,1,1,0,38.0,1,0,C
2,1,3,0,26.0,0,0,S
3,1,1,0,35.0,1,0,S
4,0,3,1,35.0,0,0,S


In [33]:
for i in data.columns:
    if i!= "Embarked":
        data[i] = (data[i] - min(data[i])) / (max(data[i])-min(data[i]))
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0.0,1.0,1.0,0.271174,0.125,0.0,S
1,1.0,0.0,0.0,0.472229,0.125,0.0,C
2,1.0,1.0,0.0,0.321438,0.000,0.0,S
3,1.0,0.0,0.0,0.434531,0.125,0.0,S
4,0.0,1.0,1.0,0.434531,0.000,0.0,S


In [34]:
new_data = data.copy()
def loaddata(data,label):
    datax = data[data[[label]].isnull().T.any()]
    for i in datax.index:
        data = data.drop(i)
    return data , datax




In [35]:
data , datax = loaddata(data,'Embarked')
ans = 0

In [36]:
def getNeighbors(trainingSet, testInstance, k):
    answer = {}
    for j in range(testInstance.shape[0]):
        distances = []
        distance0 = 0
        for x in range(trainingSet.shape[0]):
            for n in range(trainingSet.shape[1]):
                if n != trainingSet.shape[1]-1:
                    distance0 += (trainingSet.iloc[x,n]-testInstance.iloc[j,n])**2
            dist = (distance0)**0.5
            distances.append([trainingSet.iloc[x,:], dist])
        distances.sort(key = lambda x :x[-1])
        neighbors = []
        for x in range(k):
            neighbors.append(distances[x]) 
        classVotes = {}
        for i in range (len(neighbors)):
            response = neighbors [x][0][-1]
            if response in classVotes:
                classVotes[response] += 1
            else:
                classVotes[response] = 1
        sortedVotes = sorted(classVotes.items(), reverse=True)
        answer[j]=sortedVotes[0][0]
    return answer

In [37]:
neighbors=getNeighbors(data,datax,4)
neighbors[0]
for i in range(datax.shape[0]):
   datax.iloc[i,-1]=neighbors[i]
data = pd.concat([data,datax])
index01= np.arange(data.shape[0])
data.index = index01
copy.iloc[:,[-1]] = data.iloc[:,[-1]]
data = copy
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [38]:
def feature_age(age):
    if age < 18:
        return "baby"
    if age >=18 and age <= 44:
        return "teenager"
    if age >44 and age < 60:
        return "middle age"
    if age >= 60:
        return "old"

In [39]:
for i in range(data.shape[0]):
    data.iloc[i,3]=feature_age(data.iloc[i,3])
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,male,teenager,1,0,S
1,1,1,female,teenager,1,0,C
2,1,3,female,teenager,0,0,S
3,1,1,female,teenager,1,0,S
4,0,3,male,teenager,0,0,S


In [40]:
epsilon=0.001

In [41]:
def calc_ent(datasets):
    data_length = len(datasets)
    label_count = {}
    for i in range(data_length):
        label = datasets[i][0]
        if label not in label_count:
            label_count[label] = 0
        label_count[label] += 1
    ent = -sum([(p / data_length) * log(p / data_length, 2) for p in label_count.values()])
    return ent


def cond_ent(datasets, column):
    data_length = len(datasets)
    feature_sets = {}
    for i in range(data_length):
        feature = datasets[i][column]
        if feature not in feature_sets:
            feature_sets[feature] = []
        feature_sets[feature].append(datasets[i])
    cond_ent = sum([(len(p) / data_length) * calc_ent(p) for p in feature_sets.values()])
    return cond_ent


def info_gain(ent01,cond_ent01):
    return ent01 - cond_ent01


def info_gain_train(data):
    result = {}
    best_feature = []
    for i in range(data.shape[1]-1):
        ent = calc_ent(np.array(data))
        cond_ent00= cond_ent(np.array(data),i+1)
        result[data.columns[i+1]]=info_gain(ent,cond_ent00)
    result=sorted(result.items(),key=lambda x:x[1],reverse=True)
    best_feature=result[0][0]
    best_feature_info_gain=result[0][1]
    return best_feature,best_feature_info_gain


def get_feature(data):
    feature = {}
    for i in data:
        if i not in feature:
            feature[i]={}
    return feature


def max_feature(data):
    c={}
    for i in np.array(data.iloc[:,0]):
        if i not in c:
            c[i] = 0
        c[i] += 1
    d=sorted(c.items(),key=lambda x:x[1],reverse=True)
    return d[0][0]


def build_tree(data):
    tree = {}
    a,b=info_gain_train(data)
    dataset={}
    #1.若D中实例属于同一类Ck，则T为单节点树，并将类Ck作为结点的类标记，返回T
    if len(data['Survived'].value_counts()) == 1:
        return data.iloc[0,0]
    #2.若A为空，则T为单节点树，将D中实例树最大的类Ck作为该节点的类标记，返回T
    if data.shape[1]==1:
        return max_feature(data)
    #3.信息增益小于阈值eta,则置T为单节点树，并将D中是实例数最大的类Ck作为该节点的类标记，返回T
    if b < epsilon:
        return max_feature(data)
    tree[a]=get_feature(data.loc[:,a])
    for i in tree[a]:
        index = np.array(np.where(data[a]==i))[0]
        dataset[i] = data.iloc[index,:]
    data.drop(columns = a)
    for i in tree[a]:
        tree[a][i] = build_tree(dataset[i])
    return tree
    

In [42]:
tree = build_tree(data)
print(tree)

{'Sex': {'male': {'Pclass': {3: {'SibSp': {1: {'Parch': {0: {'Embarked': {'S': {'Age': {'teenager': 0, 'baby': 0}}, 'Q': {'Age': {'teenager': 0, 'baby': 1}}, 'C': 0}}, 5: 0, 1: {'Age': {'teenager': {'Embarked': {'S': 0, 'C': 1}}, 'baby': {'Embarked': {'S': 1, 'C': 0}}}}, 3: 0, 2: {'Age': {'teenager': 0, 'baby': 1}}, 4: 0}}, 0: {'Parch': {0: {'Age': {'teenager': 0, 'middle age': {'Embarked': {'S': 0, 'C': 0, 'Q': 0}}, 'old': 0, 'baby': {'Embarked': {'C': 0, 'S': 0, 'Q': 0}}}}, 2: {'Age': {'teenager': 0, 'baby': 1}}, 1: {'Age': {'teenager': 0, 'baby': 1}}}}, 3: 0, 4: {'Parch': {1: 0, 2: {'Embarked': {'S': 0, 'C': 0}}}}, 2: {'Embarked': {'C': 0, 'S': {'Age': {'teenager': 0, 'baby': 0}}}}, 5: 0, 8: 0}}, 1: {'Age': {'middle age': {'Parch': {0: {'SibSp': {0: {'Embarked': {'S': 0, 'C': 0, 'Q': 0}}, 1: 0, 2: 1}}, 1: 0, 2: 0}}, 'teenager': {'SibSp': {0: {'Parch': {0: {'Embarked': {'S': 0, 'C': 0, 'Q': 0}}, 1: {'Embarked': {'S': 0, 'Q': 0}}, 2: 0}}, 3: 0, 1: {'Parch': {0: {'Embarked': {'C': 0, '

In [43]:
from treePlotter import *
createPlot(Tree)
print(Tree)

ModuleNotFoundError: No module named 'treePlotter'